# Notebook 1
Raw data export into np.arrays stored in a HDF5 file.

### Load important modules

In [1]:
# Standard imports
import sys
import numpy as np
import pandas as pd
from numba import jit
import os
import shutil
import tables
import pickle

# Move to root directory for easier module import
os.chdir("../../")
from notebooks.server.modules.maldi_conversion import process_raw_data
from notebooks.server.modules.lookup_tables import process_lookup_tables

# multithreading/multiprocessing
from multiprocessing import Pool
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


### Create a list of raw data filenames

In [2]:
# Load filenames
l_t_names = sorted(
    [
        [
            int(name.split("MouseBrainCMC_S")[1].split("_")[0].split("A")[0].split("(")[0]),
            "/data/lipidatlas/data/data_raw/" + name + "/" + name,
        ]
        for name in os.listdir("/data/lipidatlas/data/data_raw/")
        if "MouseBrain" in name
    ]
)

# Correct for duplicates
for t_names_1, t_names_2 in zip(l_t_names[:-1], l_t_names[1:]):
    if t_names_2[0] == t_names_1[0]:
        t_names_2.append("bis")
        print("WARNING: duplicate for slice " + str(t_names_1[0]))

# Remove slices that have already been processed
path = "notebooks/server/data/temp/"
os.makedirs(path, exist_ok=True)
remove_already_loaded = False
if remove_already_loaded:
    existing_names = [int(name.split("_")[1]) for name in os.listdir(path) if "slice" in name]
    l_t_names = [x for x in l_t_names if x[0] not in existing_names]

# Print the final list of names
for t_names in l_t_names:
    print(t_names[0], t_names[1].split("/")[-1])


1 20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um
2 20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um
3 20210213_MouseBrainCMC_S3AC4_2Dpixelmode_371x195_Att25_25um
4 20210214_MouseBrainCMC_S4AD3_2Dpixelmode_354x228_Att25_25um
5 20210218_MouseBrainCMC_S5AE3_2Dpixelmode_396x272_Att25_25um
6 20210219_MouseBrainCMC_S6AE3_2Dpixelmode_423x282_Att25_25um
7 20210220_MouseBrainCMC_S7AF5_2Dpixelmode_427x263_Att25_25um
8 20210531_MouseBrainCMC_S8_duplicate_2Dpixelmode_430x285_Att30_25um
9 20210224_MouseBrainCMC_S9AH4_2Dpixelmode_467x278_Att25_25um
10 20210210_MouseBrainCMC_S10(brain2_20)_394x282_Att30_25um
11 20210301_MouseBrainCMC_S11AK5_2Dpixelmode_448x277_Att25_25um
12 20210303_MouseBrainCMC_S12AL1_2Dpixelmode_393x266_Att25_25um
13 20210304_MouseBrainCMC_S13AM1_2Dpixelmode_413x310_Att25_25um
14 20210305_MouseBrainCMC_S14AN1_2Dpixelmode_409x285_Att25_25um
15 20210313_MouseBrainCMC_S15AO2_2Dpixelmode_451x292_Att25_25um
16 20210530_MouseBrainCMC_S16_duplicate_2Dpixelmode_454

### Process raw data into numpy arrays with multiprocessing

In [3]:
multiprocessing = True
if multiprocessing:
    # Multiprocessing
    with Pool(processes=12) as pool:
        [x for x in pool.imap_unordered(process_raw_data, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_raw_data, l_t_names[3:4])]


Loading files : /data/lipidatlas/data/data_raw/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25umLoading files : /data/lipidatlas/data/data_raw/20210220_MouseBrainCMC_S7AF5_2Dpixelmode_427x263_Att25_25um/20210220_MouseBrainCMC_S7AF5_2Dpixelmode_427x263_Att25_25umLoading files : /data/lipidatlas/data/data_raw/20210224_MouseBrainCMC_S9AH4_2Dpixelmode_467x278_Att25_25um/20210224_MouseBrainCMC_S9AH4_2Dpixelmode_467x278_Att25_25umLoading files : /data/lipidatlas/data/data_raw/20210210_MouseBrainCMC_S10(brain2_20)_394x282_Att30_25um/20210210_MouseBrainCMC_S10(brain2_20)_394x282_Att30_25umLoading files : /data/lipidatlas/data/data_raw/20210531_MouseBrainCMC_S8_duplicate_2Dpixelmode_430x285_Att30_25um/20210531_MouseBrainCMC_S8_duplicate_2Dpixelmode_430x285_Att30_25umLoading files : /data/lipidatlas/data/data_raw/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_

[Warning] Not index found and build_index_from_scratch is False
[Warning] Not index found and build_index_from_scratch is False


Loading Sprectra at resolution 1e-05:   0%|          | 0/107712 [00:00<?, ?it/s]

[Warning] Not index found and build_index_from_scratch is False


Loading Sprectra at resolution 1e-05:   0%|          | 0/104538 [00:00<?, ?it/s]

...

Creating and sorting dataframes


Loading the m/z values at resolution 1e-05: 100%|██████████| 84845/84845 [01:32<00:00, 918.27it/s]


Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210530_MouseBrainCMC_S16_duplicate_2Dpixelmode_454x295_Att30_25um/20210530_MouseBrainCMC_S16_duplicate_2Dpixelmode_454x295_Att30_25um
Loading files : /data/lipidatlas/data/data_raw/20210504_MouseBrainCMC_S32_3_2Dpixelmode_298x230_Att30_25um/20210504_MouseBrainCMC_S32_3_2Dpixelmode_298x230_Att30_25um
[Warning] Not index found and build_index_from_scratch is False


Loading the m/z values at resolution 1e-05: 100%|██████████| 68540/68540 [01:14<00:00, 915.56it/s]


Creating and sorting dataframes
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Double sorting high-res array
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210412_MouseBrainCMC_S23AZ1_2Dpixelmode_360x260_Att30_25um/20210412_MouseBrainCMC_S23AZ1_2Dpixelmode_360x260_Att30_25um
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Double sorting high-res array
Creating and sorting dataframes
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210319_MouseBrainCMC_S17AQ2_2Dpixelmode_450x287_Att30_25um/20210319_MouseBrainCMC_S17AQ2_2Dpixelmode_450x287_Att30_25um
Creating and sorting dataframes
Creating and sorting dataframes
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Double sorting high-res array


### Build lookup tables

In [3]:
multiprocessing = True
if multiprocessing:
    # Multiprocessing
    with Pool(processes=12) as pool:
        [x for x in pool.map(process_lookup_tables, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_lookup_tables, l_t_names)]



Size (in mb) of lookup_table_spectra_high_res:  61.58
Shape of lookup_table_spectra_high_res:  (200, 80712)
Size (in mb) of lookup_table_spectra_high_res:  56.75
Shape of lookup_table_spectra_high_res:  (200, 74382)
Size (in mb) of lookup_table_spectra_high_res:  55.19
Shape of lookup_table_spectra_high_res:  (200, 72345)
Size (in mb) of cumulated_image_lookup_table_high_res:  56.75
Shape of cumulated_image_lookup_table_high_res:  (200, 231, 322)
Size (in mb) of cumulated_image_lookup_table_high_res:  61.58
Shape of cumulated_image_lookup_table_high_res:  (200, 228, 354)
Size (in mb) of lookup_table_spectra_high_res:  60.41
Shape of lookup_table_spectra_high_res:  (200, 79180)
Size (in mb) of lookup_table_averaged_spectrum_high_res:  0.01
Shape of lookup_table_averaged_spectrum_high_res:  (2000,)
Size (in mb) of lookup_table_averaged_spectrum_high_res:  0.01
Shape of lookup_table_averaged_spectrum_high_res:  (2000,)
Size (in mb) of cumulated_image_lookup_table_high_res:  55.19
Shape of

### Record everything as a HDF5 file and clean 

Record everything in a HDF5 file

In [4]:
output_folder = "lbae/data/whole_dataset/"
input_folder = "notebooks/server/data/temp/"
os.makedirs(output_folder, exist_ok=True)

dic_slices = {}
# Open the hdf5 file
with tables.open_file(output_folder + 'slices.hdf5', mode="w") as hdf5_file:
    # Loop over slice files
    for slice_name in os.listdir(input_folder):

        # Load slice arrays
        npzfile = np.load(input_folder + slice_name)
        array_pixel_indexes_high_res = npzfile["array_pixel_indexes_high_res"]
        array_spectra_high_res = npzfile["array_spectra_high_res"]
        array_averaged_mz_intensity_low_res = npzfile["array_averaged_mz_intensity_low_res"]
        array_averaged_mz_intensity_high_res = npzfile["array_averaged_mz_intensity_high_res"]
        image_shape = npzfile["image_shape"]
        divider_lookup = npzfile["divider_lookup"]
        lookup_table_spectra_high_res = npzfile["lookup_table_spectra_high_res"]
        cumulated_image_lookup_table_high_res = npzfile["cumulated_image_lookup_table_high_res"]
        lookup_table_averaged_spectrum_high_res = npzfile["lookup_table_averaged_spectrum_high_res"]

        # Create a new group in the hdf5 file
        slice_index = slice_name.split("_")[1]
        group = hdf5_file.create_group("/", "s" + slice_index, "Slice " + slice_index)

        # Register heavy arrays and lookup table in the newly created group
        hdf5_file.create_array(group, "array_spectra", array_spectra_high_res)
        hdf5_file.create_array(group, "array_avg_intensity", array_averaged_mz_intensity_high_res)
        hdf5_file.create_array(group, "array_lookup_mz", lookup_table_spectra_high_res)
        hdf5_file.create_array(group, "array_cumulated_lookup_mz_image", cumulated_image_lookup_table_high_res)

        # Register the rest in a pickle dictionnary
        dic_slices["s" + slice_index] = {
            "image_shape": image_shape,
            "divider_lookup": divider_lookup,
            "array_avg_intensity_downsampled": array_averaged_mz_intensity_low_res,
            "array_lookup_pixels": array_pixel_indexes_high_res,
            "array_lookup_mz_avg": lookup_table_averaged_spectrum_high_res,
        }

        # Pytables is not happy with recording very low rows (m/z values and intensities) but taking the transpose means
        # that loading just the m/z values or intensities will jump every odd/even index... Making it very slow.

with open(output_folder + 'slices.pickle', 'wb') as handle:
    pickle.dump(dic_slices, handle)

/home/colas/miniconda3/lib/python3.9/site-packages/tables/leaf.py:402: PerformanceWarning: The Leaf ``/s10/array_spectra`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  warnings.warn("""\
/home/colas/miniconda3/lib/python3.9/site-packages/tables/leaf.py:402: PerformanceWarning: The Leaf ``/s11/array_spectra`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have s

Test that everything went alright

In [10]:
with tables.open_file(output_folder + 'slices.hdf5', mode='r') as hdf5_file:
    n_slices = len(list(hdf5_file.root))
    print('Number of slices recorder: ', n_slices)
    print('Array spectra of slice 1: ', hdf5_file.root['s1'].array_spectra)
    print('List all arrays recorded in the file: ')
    for group in hdf5_file.walk_groups():
        for array in hdf5_file.list_nodes(group, classname='Array'):
            print(array)

Number of slices recorder:  32
Array spectra of slice 1:  /s1/array_spectra (Array(2, 102305653)) ''
List all arrays recorded in the file: 
/s1/array_avg_intensity (Array(2, 1021129)) ''
/s1/array_cumulated_lookup_mz_image (Array(200, 231, 322)) ''
/s1/array_lookup_mz (Array(200, 74382)) ''
/s1/array_spectra (Array(2, 102305653)) ''
/s10/array_avg_intensity (Array(2, 1898870)) ''
/s10/array_cumulated_lookup_mz_image (Array(200, 282, 394)) ''
/s10/array_lookup_mz (Array(200, 111108)) ''
/s10/array_spectra (Array(2, 151226720)) ''
/s11/array_avg_intensity (Array(2, 2262659)) ''
/s11/array_cumulated_lookup_mz_image (Array(200, 277, 448)) ''
/s11/array_lookup_mz (Array(200, 124096)) ''
/s11/array_spectra (Array(2, 216433078)) ''
/s12/array_avg_intensity (Array(2, 1793930)) ''
/s12/array_cumulated_lookup_mz_image (Array(200, 266, 393)) ''
/s12/array_lookup_mz (Array(200, 104538)) ''
/s12/array_spectra (Array(2, 146793259)) ''
/s13/array_avg_intensity (Array(2, 466769)) ''
/s13/array_cumulat

Clean temporary folder

In [5]:
clean = True
if clean:
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
